In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

import data
import lightgbm as lgb
import numpy as np
import os
import sys
import pandas as pd
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import mean_squared_error, roc_auc_score, accuracy_score

from model import MODEL

%matplotlib inline

In [4]:
train_ad, train_click, train_user, test_ad, test_click = data.get_data()
train_user, valid_user = train_test_split(train_user, test_size=0.33, random_state=42)
train_record = data.get_part_click(train_click, train_user)
valid_record = data.get_part_click(train_click, valid_user)

# train_record
train_record = pd.merge(train_record, train_ad, on="creative_id")
# valid_record
valid_record = pd.merge(valid_record, train_ad, on="creative_id")

In [5]:
train_features, train_age, train_gender = data.split_feature_target(train_record)
valid_features, valid_age, valid_gender = data.split_feature_target(train_record)

train_features = train_features.values
train_age = train_age.values - 1
train_gender = train_gender.values - 1

valid_features = valid_features.values
valid_age = valid_age.values - 1
valid_gender = valid_gender.values - 1

In [6]:
lgb_traindata_gender = lgb.Dataset(train_features, train_gender)
lgb_traindata_age = lgb.Dataset(train_features, train_age)

lgb_valdata_gender = lgb.Dataset(valid_features, valid_gender, reference=lgb_traindata_gender)
lgb_valdata_age = lgb.Dataset(valid_features, valid_age, reference=lgb_traindata_age)

In [8]:
gender_model = MODEL(model_kind="gender")
gender_model.train(lgb_traindata_gender, lgb_valdata_gender)
gender_model.save_model()

/home/yangshanghui/anaconda3/envs/taac/lib/python3.7/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [1, 3, 4, 5, 6, 7]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[1]	valid_0's binary_logloss: 0.611592	valid_0's auc: 0.697599
Training until validation scores don't improve for 10 rounds
[2]	valid_0's binary_logloss: 0.600938	valid_0's auc: 0.708567
[3]	valid_0's binary_logloss: 0.592201	valid_0's auc: 0.712971
[4]	valid_0's binary_logloss: 0.584996	valid_0's auc: 0.714646
[5]	valid_0's binary_logloss: 0.57863	valid_0's auc: 0.717341
[6]	valid_0's binary_logloss: 0.573297	valid_0's auc: 0.721578
[7]	valid_0's binary_logloss: 0.568651	valid_0's auc: 0.723846
[8]	valid_0's binary_logloss: 0.564491	valid_0's auc: 0.72479
[9]	valid_0's binary_logloss: 0.561588	valid_0's auc: 0.725633
[10]	valid_0's binary_logloss: 0.558431	valid_0's auc: 0.726099
[11]	valid_0's binary_logloss: 0.556082	valid_0's auc: 0.727152
[12]	valid_0's binary_logloss: 0.553333	valid_0's auc: 0.728807
[13]	valid_0's binary_logloss: 0.551086	valid_0's auc: 0.729338
[14]	valid_0's binary_logloss: 0.548917	valid_0's auc: 0.730484
[15]	valid_0's binary_logloss: 0.547002	valid_0's auc:

In [9]:
test_record = pd.merge(test_click, test_ad, on="creative_id")

In [10]:
test_features = test_record.iloc[:, [0, 2, 3, 4, 5, 6, 7, 8]]
test_features = test_features.values

In [11]:
test_pred = gender_model.predict(test_features)

In [12]:
test_pred

array([0.76126466, 0.76126466, 0.76126466, ..., 0.32078798, 0.34056228,
       0.31826297])